In [123]:
from pathlib import Path

import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
from math import radians, cos, sin, asin, sqrt
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold,  KFold,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


pd.set_option("display.max_columns", 100)

from google.colab import drive
drive.mount('/content/gdrive')
DATA_PATH = "/content/gdrive/My Drive/MLdata/"

train_df = pd.read_csv(
    DATA_PATH + "train.csv", 
    index_col="tripid"
)
test_df = pd.read_csv(
    DATA_PATH + "test.csv", 
    index_col="tripid"
)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Data Cleaning

In [124]:
train_df["label"] = train_df["label"].map({"incorrect": 0, "correct":1})

In [125]:
train_df["pickup"] = pd.to_datetime(train_df["pickup_time"])
train_df["drop"] = pd.to_datetime(train_df["drop_time"])
train_df ['difference'] = train_df["drop"] - train_df["pickup"]
train_df ['difference'] = train_df ['difference']/np.timedelta64(1,'s')
train_df['duration'] = train_df['duration'].fillna(train_df ['difference'])
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   additional_fare            16974 non-null  float64       
 1   duration                   17176 non-null  float64       
 2   meter_waiting              16974 non-null  float64       
 3   meter_waiting_fare         16974 non-null  float64       
 4   meter_waiting_till_pickup  16974 non-null  float64       
 5   pickup_time                17176 non-null  object        
 6   drop_time                  17176 non-null  object        
 7   pick_lat                   17176 non-null  float64       
 8   pick_lon                   17176 non-null  float64       
 9   drop_lat                   17176 non-null  float64       
 10  drop_lon                   17176 non-null  float64       
 11  fare                       17039 non-null  float64     

In [126]:
train_df.head()

# new_df = train_df["pickup_time"]-train_df["drop_time"]
# train_df.info()
# train_df.head()
train_df.isnull().sum()
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,pickup,drop,difference
tripid,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1,2019-11-01 00:20:00,2019-11-01 00:34:00,840.0
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1,2019-11-01 00:56:00,2019-11-01 01:09:00,780.0
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1,2019-11-01 01:08:00,2019-11-01 01:26:00,1080.0
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1,2019-11-01 02:27:00,2019-11-01 02:37:00,600.0
189128020,NaN,1020.0,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1,2019-11-01 03:34:00,2019-11-01 03:51:00,1020.0


In [127]:
train_df2 = train_df.drop(['pickup_time','drop_time','pickup','drop','difference'],axis='columns')
train_df2.isnull().sum()

additional_fare              202
duration                       0
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [128]:
train_df2['fare'] = train_df2['fare'] .fillna(train_df2.fare.mean())

In [129]:
train_df2['additional_fare'] = train_df2['additional_fare'].fillna(train_df2.additional_fare.mean())

In [130]:
train_df2['meter_waiting'] = train_df2['meter_waiting'].fillna(train_df2.meter_waiting.mean())

In [131]:
train_df2['meter_waiting_fare'] = train_df2['meter_waiting_fare'].fillna(train_df2.meter_waiting_fare.mean())

In [132]:
train_df2['meter_waiting_till_pickup'] = train_df2['meter_waiting_till_pickup'].fillna(train_df2.meter_waiting_till_pickup.mean())

In [133]:
train_df2.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
dtype: int64

In [134]:
train_df2['lat_change'] = abs(train_df2['pick_lat']- train_df2['drop_lat'])
train_df2['lon_change'] = abs(train_df2['pick_lon']- train_df2['drop_lon'])
train_df2.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,lat_change,lon_change
tripid,,,,,,,,,,,,,
189123628,10.500000,834.0,56.000000,0.000000,64.000000,6.86252,79.8993,6.90330,79.8783,270.32,1,0.04078,0.0210
189125358,10.500000,791.0,47.000000,0.000000,134.000000,6.88589,79.8984,6.91373,79.8923,197.85,1,0.02784,0.0061
189125719,10.500000,1087.0,80.000000,0.000000,61.000000,6.90839,79.8651,6.93669,79.9146,301.64,1,0.02830,0.0495
189127273,10.500000,598.0,271.000000,15.663800,68.000000,6.92570,79.8895,6.92748,79.8971,82.30,1,0.00178,0.0076
189128020,13.719651,1020.0,629.074231,32.057666,112.466832,6.87441,79.8615,6.84478,79.9290,358.39,1,0.02963,0.0675


In [137]:
from math import radians, cos, sin, asin, sqrt
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # 6371 is Radius of earth in kilometers. Use 3956 for miles
    return km

train_df2['distance'] = haversine_np(train_df2['pick_lat'], train_df2['pick_lon'], train_df2['drop_lat'] , train_df2['drop_lon'])

In [138]:
clean_df = train_df2.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

In [139]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            17176 non-null  float64
 1   duration                   17176 non-null  float64
 2   meter_waiting              17176 non-null  float64
 3   meter_waiting_fare         17176 non-null  float64
 4   meter_waiting_till_pickup  17176 non-null  float64
 5   fare                       17176 non-null  float64
 6   label                      17176 non-null  int64  
 7   lat_change                 17176 non-null  float64
 8   lon_change                 17176 non-null  float64
 9   distance                   17176 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 1.4 MB


In [140]:
clean_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,lat_change,lon_change,distance
tripid,,,,,,,,,,
189123628,10.500000,834.0,56.000000,0.000000,64.000000,270.32,1,0.04078,0.0210,2.467063
189125358,10.500000,791.0,47.000000,0.000000,134.000000,197.85,1,0.02784,0.0061,0.868942
189125719,10.500000,1087.0,80.000000,0.000000,61.000000,301.64,1,0.02830,0.0495,5.531798
189127273,10.500000,598.0,271.000000,15.663800,68.000000,82.30,1,0.00178,0.0076,0.845795
189128020,13.719651,1020.0,629.074231,32.057666,112.466832,358.39,1,0.02963,0.0675,7.527884


Training

In [141]:
X_train = clean_df.drop(['label'],axis='columns')
Y_train = clean_df['label']
Y_train.head()

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
Name: label, dtype: int64

In [142]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_features = clean_df.select_dtypes(include=['int64', 'float64']).drop(['label'], axis=1).columns
categorical_features = clean_df.select_dtypes(include=['object']).columns
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [119]:
#test with different classifiers
knn = KNeighborsClassifier(n_neighbors=10, p=2,metric='euclidean')
rft = RandomForestClassifier(n_jobs=2,random_state=0)
LR = LogisticRegression(random_state = 2)
xgb= XGBClassifier( learning_rate =0.1, n_estimators=500)

In [46]:
kfold = StratifiedKFold(n_splits=10)

In [47]:
random_state = 2

In [71]:
c_results = []
c_results.append(cross_val_score(rft, X_train, y = Y_train, scoring = 'f1', cv = kfold, n_jobs=4)) 

c_means = 0
c_std = 0
for c_result in c_results:
    c_means = c_result.mean()
    c_std  = c_result.std()

c_means

0.9711730543729032

In [143]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', rft)])

In [145]:
x_train, x_eval, y_eval, y_eval = train_test_split(X_train, Y_train, test_size=0.25,stratify=Y_train)

In [148]:
pipeline.fit(x_eval, y_eval)
y_pred = pipeline.predict(x_eval)

In [149]:
f1_score(y_eval, y_pred,average='weighted')

1.0

In [144]:
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                               

Predicting

In [150]:
test_df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64

In [151]:
test_df.info()
test_df.head()
test_df.isnull().sum()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8576 entries, 213284604 to 222860703
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            8576 non-null   float64
 1   duration                   8576 non-null   int64  
 2   meter_waiting              8576 non-null   int64  
 3   meter_waiting_fare         8576 non-null   float64
 4   meter_waiting_till_pickup  8576 non-null   int64  
 5   pickup_time                8576 non-null   object 
 6   drop_time                  8576 non-null   object 
 7   pick_lat                   8576 non-null   float64
 8   pick_lon                   8576 non-null   float64
 9   drop_lat                   8576 non-null   float64
 10  drop_lon                   8576 non-null   float64
 11  fare                       8576 non-null   float64
dtypes: float64(7), int64(3), object(2)
memory usage: 871.0+ KB


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [152]:
test_df['lat_change'] = abs(test_df['pick_lat']- test_df['drop_lat'])
test_df['lon_change'] = abs(test_df['pick_lon']- test_df['drop_lon'])
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,lat_change,lon_change
tripid,,,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27,0.05964,0.0090
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70,0.36077,0.0983
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00,0.01606,0.0511
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32,0.02968,0.0009
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47,0.00907,0.0216


In [153]:
test_df['distance'] = haversine_np(test_df['pick_lat'], test_df['pick_lon'], test_df['drop_lat'] , test_df['drop_lon'])

In [154]:
test_df2 = test_df.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

test_df2.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
fare                         0
lat_change                   0
lon_change                   0
distance                     0
dtype: int64

In [156]:
result = pipeline.predict(test_df2)
test_df2['prediction'] = result

In [157]:
final_df = test_df2.drop(['additional_fare','duration','meter_waiting_till_pickup','fare','lat_change','lon_change','meter_waiting','meter_waiting_fare'],axis='columns')

final_df.head()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8576 entries, 213284604 to 222860703
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   distance    8576 non-null   float64
 1   prediction  8576 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 201.0 KB


In [ ]:
final_df.to_csv('submission.csv', index=True)